In [53]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [47]:
import requests

url = 'https://raw.githubusercontent.com/ckirby19/league-of-legends-player-profiles/api_data/json_data_test.json'
response = requests.get(url)
d = response.json()

In [48]:
frames = d['info']['frames']

In [49]:
len(frames)

30

In [50]:
def compute_totals(frame, metric="totalGold"):
    team1 = sum(p[metric] for pid, p in frame["participantFrames"].items() if int(pid) <= 5)
    team2 = sum(p[metric] for pid, p in frame["participantFrames"].items() if int(pid) > 5)
    return team1, team2

def pythagorean_expectation(s1, s2, alpha=1.5):
    denominator = (s1 ** alpha + s2 ** alpha)
    if denominator == 0:
        return 0.5 
    return (s1 ** alpha) / denominator

results = []
for frame in frames:
    t = frame["timestamp"]
    g1, g2 = compute_totals(frame, "totalGold")
    x1, x2 = compute_totals(frame, "xp")

    p_gold = pythagorean_expectation(g1, g2)
    p_xp   = pythagorean_expectation(x1, x2)
    p_final = (p_gold + p_xp) / 2  # simple average

    results.append({
        "time": t,
        "gold_advantage": g1 - g2,
        "xp_advantage": x1 - x2,
        "p_win_gold": p_gold,
        "p_win_xp": p_xp,
        "p_final": p_final
    })

# Example: print first few
for r in results:
    print(r)

{'time': 0, 'gold_advantage': 0, 'xp_advantage': 0, 'p_win_gold': 0.5, 'p_win_xp': 0.5, 'p_final': 0.5}
{'time': 60001, 'gold_advantage': 0, 'xp_advantage': 0, 'p_win_gold': 0.5, 'p_win_xp': 0.5, 'p_final': 0.5}
{'time': 120019, 'gold_advantage': 98, 'xp_advantage': 298, 'p_win_gold': 0.512584139871822, 'p_win_xp': 0.6166753125648154, 'p_final': 0.5646297262183188}
{'time': 180025, 'gold_advantage': -95, 'xp_advantage': -15, 'p_win_gold': 0.49220579875420256, 'p_win_xp': 0.4984608044524197, 'p_final': 0.4953333016033111}
{'time': 240058, 'gold_advantage': 281, 'xp_advantage': 73, 'p_win_gold': 0.5172806830623874, 'p_win_xp': 0.5048122581349023, 'p_final': 0.5110464705986448}
{'time': 300104, 'gold_advantage': 522, 'xp_advantage': 540, 'p_win_gold': 0.5254066154666989, 'p_win_xp': 0.5251528573466913, 'p_final': 0.525279736406695}
{'time': 360124, 'gold_advantage': 1639, 'xp_advantage': 535, 'p_win_gold': 0.5639386947576625, 'p_win_xp': 0.52027024250378, 'p_final': 0.5421044686307213}
{'

In [51]:
ALPHA = 1.5

frames = d["info"]["frames"]
frame_interval_ms = d["info"].get("frameInterval", 60000)

def compute_team_totals(frame, metric: str):
    t1 = 0
    t2 = 0
    for pid_str, pf in frame["participantFrames"].items():
        pid = int(pid_str)
        val = pf.get(metric, 0) or 0
        if pid <= 5:
            t1 += val
        else:
            t2 += val
    return t1, t2

def pythag_expectation(s1, s2, alpha=ALPHA):
    # Guard against degenerate cases
    s1a = float(s1)**alpha if s1 > 0 else 0.0
    s2a = float(s2)**alpha if s2 > 0 else 0.0
    denom = s1a + s2a
    if denom == 0:
        return 0.5
    return s1a / denom

rows = []
for f in frames:
    t_ms = f.get("timestamp", 0)
    t_min = t_ms / 60000.0  # convert ms -> minutes
    
    g1, g2 = compute_team_totals(f, "totalGold")
    x1, x2 = compute_team_totals(f, "xp")
    
    p_gold = pythag_expectation(g1, g2, ALPHA)
    p_xp   = pythag_expectation(x1, x2, ALPHA)
    p_final = (p_gold + p_xp) / 2.0
    
    rows.append({
        "time_min": t_min,
        "gold_team1": g1,
        "gold_team2": g2,
        "gold_adv": g1 - g2,
        "xp_team1": x1,
        "xp_team2": x2,
        "xp_adv": x1 - x2,
        "p_win_gold": p_gold,
        "p_win_xp": p_xp,
        "p_win_final": p_final
    })

df = pd.DataFrame(rows)

In [43]:
display(df.head())

,time_min,gold_team1,gold_team2,gold_adv,xp_team1,xp_team2,xp_adv,p_win_gold,p_win_xp,p_win_final
0,0.000000,2500,2500,0,0,0,0,0.500000,0.500000,0.500000
1,1.000017,2500,2500,0,0,0,0,0.500000,0.500000,0.500000
2,2.000317,2969,2871,98,1097,799,298,0.512584,0.616675,0.564630
3,3.000417,4523,4618,-95,3647,3662,-15,0.492206,0.498461,0.495333
4,4.000967,6237,5956,281,5725,5652,73,0.517281,0.504812,0.511046


In [60]:
# Win Probability
fig_wp = go.Figure()
fig_wp.add_trace(go.Scatter(
    x=df["time_min"], y=df["p_win_final"],
    mode="lines",
    line=dict(width=4, color="#F2B705"),
    line_shape="spline",
    name="P(win) – Team 1 (Gold+XP)"
))
fig_wp.add_hline(y=0.5, line_width=1, line_dash="dash", line_color="rgba(255,255,255,0.6)")

fig_wp.update_layout(
    title="Team 1 Win Probability (Gold & XP Pythagorean Blend)",
    template="plotly_dark",
    paper_bgcolor="#1f1f1f", plot_bgcolor="#1f1f1f",
    xaxis_title="Time (min)", yaxis_title="P(win)",
    yaxis=dict(range=[0.45, 0.60], tickformat=".0%"),
    hovermode="x unified",
    margin=dict(l=60, r=30, t=60, b=50)
)
fig_wp.update_xaxes(showgrid=True, gridcolor="rgba(255,255,255,0.08)")
fig_wp.update_yaxes(showgrid=True, gridcolor="rgba(255,255,255,0.08)")

# Gold & XP Advantage
fig_adv = go.Figure()
fig_adv.add_trace(go.Scatter(
    x=df["time_min"], y=df["gold_adv"],
    mode="lines", line=dict(width=3, color="#F2B705"),
    line_shape="spline", name="Gold Advantage (T1 - T2)"
))
fig_adv.add_trace(go.Scatter(
    x=df["time_min"], y=df["xp_adv"],
    mode="lines", line=dict(width=3, color="#1E90FF"),
    line_shape="spline", name="XP Advantage (T1 - T2)"
))
fig_adv.add_hline(y=0, line_width=1, line_dash="dash", line_color="rgba(255,255,255,0.6)")

fig_adv.update_layout(
    title="Team 1 Advantages Over Time",
    template="plotly_dark",
    paper_bgcolor="#1f1f1f", plot_bgcolor="#1f1f1f",
    xaxis_title="Time (min)", yaxis_title="Advantage",
    hovermode="x unified",
    margin=dict(l=60, r=30, t=60, b=50)
)
fig_adv.update_xaxes(showgrid=True, gridcolor="rgba(255,255,255,0.08)")
fig_adv.update_yaxes(showgrid=True, gridcolor="rgba(255,255,255,0.08)")

# ---------- Show ----------
fig_wp.show()
fig_adv.show()

In [59]:
# Save as HTML
fig_wp.write_html("figs/winprob_dark.html", include_plotlyjs="cdn")
fig_adv.write_html("figs/advantages_dark.html", include_plotlyjs="cdn")

In [61]:
events = [event for frame in frames for event in frame['events']]

event_df = pd.DataFrame(events)

In [62]:
display(event_df)

,realTimestamp,timestamp,type,itemId,participantId,afterId,beforeId,goldGain,levelUpType,skillSlot,...,laneType,teamId,killerTeamId,monsterSubType,monsterType,buildingType,towerType,actualStartTime,gameId,winningTeam
0,1.712859e+12,0,PAUSE_END,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,2036,ITEM_PURCHASED,3865.0,10.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,2036,ITEM_PURCHASED,2003.0,10.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,2036,ITEM_PURCHASED,2003.0,10.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,2336,ITEM_PURCHASED,1102.0,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
951,NaN,1727532,CHAMPION_SPECIAL_KILL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
952,NaN,1727532,LEVEL_UP,NaN,4.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
953,NaN,1727797,CHAMPION_KILL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
954,NaN,1727797,CHAMPION_SPECIAL_KILL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
